In [15]:
import numpy as np
import pandas as pd

import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import csv
import re
import random

2024-01-17 13:58:23.931271: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 13:58:23.931448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 13:58:24.114395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 13:58:24.381364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 13:58:30.829503: W tensorflow/compiler/tf2

In [4]:
output_kikuyu_file = '../dataset/Kikuyu/kikuyu.txt'
output_swahili_file = '../dataset/Kikuyu/swahili.txt'


# Defining lines as a list of each line
with open(output_kikuyu_file, 'r', encoding='utf-8') as f:
  lines = f.read().strip().split('\n')
with open(output_swahili_file, 'r', encoding='utf-8') as f:
  lines2 = f.read().strip().split('\n')

lines = [" ".join(re.findall(r"[A-Za-z0-9]+",line)) for line in lines]
lines2 = [" ".join(re.findall(r"[A-Za-z0-9]+",line)) for line in lines2]

In [5]:
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
# pairs = pairs[:3000]
random.shuffle(pairs)
print(len(pairs))

3500


In [9]:
print(pairs[910])

('M rongo r na inya', 'Ishirini na nne')


In [10]:
# printing random pairs
print(random.choice(pairs))

('Kwoguo Aburamu ak amb ra hema yake', 'Kwa hiyo Abramu akangoa hema')


In [11]:
# shuffling the pairs
random.shuffle(pairs)
num_val_samples = int(0.15 * len(pairs))
num_train_samples = len(pairs) - 2 * num_val_samples
train_pairs = pairs[:num_train_samples]
val_pairs = pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs =pairs[num_train_samples + num_val_samples:]
print(len(train_pairs))

2450


In [14]:
train_pairs[1770]

('naake Ngai ak honia Abimeleki', 'naye Mungu akamponya Abimeleki')

## Vectorizing the Kikuyu and Swahili text pairs


In [16]:

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_kikuyu_texts = [pair[0] for pair in train_pairs]
train_swahili_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_kikuyu_texts)
target_vectorization.adapt(train_swahili_texts)


## Preparing datasets for the translation task


In [18]:

batch_size = 512

def format_dataset(kik, swa):
    kik = source_vectorization(kik)
    swa = target_vectorization(swa)
    return ({
        "kikuyu": kik,
        "swahili": swa[:, :-1],
    }, swa[:, 1:])

def make_dataset(pairs):
    kik_texts, swa_texts = zip(*pairs)
    kik_texts = list(kik_texts)
    swa_texts = list(swa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((kik_texts, swa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [19]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['kikuyu'].shape: {inputs['kikuyu'].shape}")
    print(f"inputs['swahili'].shape: {inputs['swahili'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['kikuyu'].shape: (402, 20)
inputs['swahili'].shape: (402, 20)
targets.shape: (402, 20)


2024-01-17 14:01:57.291568: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Sequence-to-sequence learning with RNNs GRU-based encoder

In [20]:
# Sequence-to-sequence learning with RNNs
# GRU-based encoder

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="kikuyu")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

## GRU-based decoder and the end-to-end model


In [21]:

past_target = keras.Input(shape=(None,), dtype="int64", name="swahili")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

2024-01-17 14:03:20.921343: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61440000 exceeds 10% of free system memory.
2024-01-17 14:03:21.034235: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61440000 exceeds 10% of free system memory.
2024-01-17 14:03:21.099931: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61440000 exceeds 10% of free system memory.


## Training our recurrent sequence-to-sequence model


In [23]:

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
# seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)
history=seq2seq_rnn.fit(train_ds, epochs=20, validation_data=val_ds)


Epoch 1/20
5/5 [==============================] - 215s 38s/step - loss: 8.7797 - accuracy: 0.1831 - val_loss: 7.8899 - val_accuracy: 0.1853
Epoch 2/20
5/5 [==============================] - 158s 33s/step - loss: 7.6734 - accuracy: 0.1831 - val_loss: 7.0717 - val_accuracy: 0.1853
Epoch 3/20
5/5 [==============================] - 154s 33s/step - loss: 6.8277 - accuracy: 0.1831 - val_loss: 6.9563 - val_accuracy: 0.1853
Epoch 4/20
5/5 [==============================] - 139s 29s/step - loss: 6.6382 - accuracy: 0.1831 - val_loss: 6.7208 - val_accuracy: 0.1853
Epoch 5/20
5/5 [==============================] - 104s 22s/step - loss: 6.4826 - accuracy: 0.1831 - val_loss: 6.6909 - val_accuracy: 0.1853
Epoch 6/20
5/5 [==============================] - 116s 25s/step - loss: 6.3912 - accuracy: 0.1831 - val_loss: 6.6105 - val_accuracy: 0.1853
Epoch 7/20
5/5 [==============================] - 115s 25s/step - loss: 6.3202 - accuracy: 0.1847 - val_loss: 6.6052 - val_accuracy: 0.1975
Epoch 8/20
5/5 [====

## Translating new sentences with our RNN encoder and decoder


In [24]:

swa_vocab = target_vectorization.get_vocabulary()
swa_index_lookup = dict(zip(range(len(swa_vocab)), swa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = swa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_kik_texts = [pair[0] for pair in pairs]
for _ in range(20):
    input_sentence = random.choice(test_kik_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Naake m th nj ri Ngai am tware nyoni iyo k gongoonain
1/1 [==============================] - 0s 188ms/step
[start] na na na na                
-
n ng m he b r ri wothe cio roona
1/1 [==============================] - 0s 239ms/step
[start] na na na                 
-
Nd r r irotu ka ndungata ciaku
1/1 [==============================] - 0s 203ms/step
[start] na na                  
-
Wee uri muhinga hari nii
1/1 [==============================] - 0s 233ms/step
[start] na                   
-
No Jesu akimwira atiri
1/1 [==============================] - 0s 198ms/step
[start]                    
-
Nake petero akimutwara akiambiriria kumukaania
1/1 [==============================] - 0s 219ms/step
[start] na                   
-
Harani aar ithe wa Loti
1/1 [==============================] - 0s 190ms/step
[start] na                   
-
Acooke a it r rie maguta
1/1 [==============================] - 0s 232ms/step
[start] na na                  
-

1/1 [==============================] - 0s 2